In [1]:
from mypackage import scraping_functions as sf
from mypackage import predict_functions as pf
from mypackage import cleaning_functions as cf
import pandas as pd
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup, Comment
import re
import time
from math import inf

In [ ]:
game_url = 'https://www.pro-football-reference.com/boxscores/202309070kan.htm'
df = sf.scrape_game_data(game_url)
train_data = pd.read_csv('../nfl_scraping_project/mypackage/data/week_10_2023_plays.csv')

In [ ]:
# Prepare Training Data
train_data['seconds_left'] = train_data.apply(cf.seconds_left, axis=1)
train_data['score_diff'] = train_data.iloc[:,5].astype(float) - train_data.iloc[:,6].astype(float)
train_data['adjusted_score'] = train_data.apply(cf.adjusted_score_calc, axis=1)
train_data['win_team'] = train_data.apply(lambda row: cf.win(pd.DataFrame(row).transpose()), axis=1)

y_train = train_data['win_team']
y_train = y_train.apply(lambda x:float(x[0]))

columns_to_drop = [5, 6]
additional_columns_to_drop = ['Quarter', 'Detail', 'Location', 'Time', 'win_team', 'play_start_time', 'game_id', 'win_team']

train_data = train_data.drop(columns=train_data.columns[columns_to_drop])
train_data = train_data.drop(columns=additional_columns_to_drop)
train_data['field_side'] = pd.Categorical(train_data['field_side'])
train_data['Play_Type'] = pd.Categorical(train_data['Play_Type'])
train_data['possession'] = pd.Categorical(train_data['possession'])
train_data['Down'] = pd.Categorical(train_data['Down'])
train_data['ToGo'] = pd.to_numeric(train_data['ToGo'], errors='coerce')
train_data['ToGo'] = train_data['ToGo'].astype(float)
train_data['EPB'] = train_data['EPB'].astype(float)
train_data['EPA'] = train_data['EPA'].astype(float)
train_data = pd.get_dummies(train_data, columns=['Play_Type', 'Down', 'field_side', 'possession'])

train_data = train_data.fillna(0)

In [ ]:
train_data

In [2]:
game_links = [
    'https://www.pro-football-reference.com/boxscores/202309110nyj.htm',
    'https://www.pro-football-reference.com/boxscores/202309070kan.htm',
    'https://www.pro-football-reference.com/boxscores/202309100atl.htm',
    'https://www.pro-football-reference.com/boxscores/202309100chi.htm',
    'https://www.pro-football-reference.com/boxscores/202309100cle.htm',
    'https://www.pro-football-reference.com/boxscores/202309100clt.htm',
    'https://www.pro-football-reference.com/boxscores/202309100den.htm',
    'https://www.pro-football-reference.com/boxscores/202309100min.htm',
    'https://www.pro-football-reference.com/boxscores/202309100nor.htm',
    'https://www.pro-football-reference.com/boxscores/202309100nwe.htm',
    'https://www.pro-football-reference.com/boxscores/202309100nyg.htm',
    'https://www.pro-football-reference.com/boxscores/202309100pit.htm',
    'https://www.pro-football-reference.com/boxscores/202309100rav.htm',
    'https://www.pro-football-reference.com/boxscores/202309100sdg.htm',
    'https://www.pro-football-reference.com/boxscores/202309100sea.htm',
    'https://www.pro-football-reference.com/boxscores/202309100was.htm'    
    ]

game_file_path = '../nfl_scraping_project/mypackage/data/week_10_2023_games.csv'
data_file_path = '../nfl_scraping_project/mypackage/data/week_10_2023_plays.csv'

sf.scrape_games(game_links, game_file_path, data_file_path)

Jets vs. Bills 
 total plays: 123
Chiefs vs. Lions 
 total plays: 132
Falcons vs. Panthers 
 total plays: 125
Bears vs. Packers 
 total plays: 131
Browns vs. Bengals 
 total plays: 128
Colts vs. Jaguars 
 total plays: 128
Broncos vs. Raiders 
 total plays: 126
Vikings vs. Buccaneers 
 total plays: 132
Saints vs. Titans 
 total plays: 130
Patriots vs. Eagles 
 total plays: 140
Giants vs. Cowboys 
 total plays: 122
Steelers vs. 49ers 
 total plays: 132
Ravens vs. Texans 
 total plays: 139
Chargers vs. Dolphins 
 total plays: 142
Seahawks vs. Rams 
 total plays: 134
Commanders vs. Cardinals 
 total plays: 129
done


In [3]:
team_keys = {
    'DET': 'Lions',
    'KAN': 'Chiefs',
    'ATL': 'Falcons',
    'CAR': 'Panthers',
    'CLE': 'Browns', 
    'CIN': 'Bengals', 
    'IND': 'Colts', 
    'JAX': 'Jaguars',
    'MIN': 'Vikings', 
    'TAM': 'Buccaneers',
    'NOR': 'Saints', 
    'TEN': 'Titans',
    'PIT': 'Steelers', 
    'SFO': '49ers', 
    'BAL': 'Ravens',
    'HOU': 'Texans', 
    'WAS': 'Commanders',
    'ARI': 'Cardinals',
    'CHI': 'Bears', 
    'GNB': 'Packers',
    'DEN': 'Broncos', 
    'LVR': 'Raiders', 
    'NWE': 'Patriots', 
    'PHI': 'Eagles', 
    'LAC': 'Chargers', 
    'MIA': 'Dolphins',
    'SEA': 'Seahawks',
    'LAR': 'Rams',
    'NYG': 'Giants',
    'DAL': 'Cowboys',
    'NYJ': 'Jets', 
    'BUF': 'Bills'
}
pbp_data = []
game_url = 'https://www.pro-football-reference.com/boxscores/202309110nyj.htm'
r = requests.get(game_url)
game_page_soup = BeautifulSoup(r.text, 'html.parser')
# scraping drive data for home and away team
home_drives = sf.get_drive_table('home', game_page_soup)
vis_drives = sf.get_drive_table('vis', game_page_soup)
# getting home and away team variables
home_team = home_drives['team'][0]
vis_team = vis_drives['team'][0]
home_vis = {home_team: 'home', vis_team: 'away'}

teams = [home_team, vis_team]
drives = pd.concat([home_drives, vis_drives], axis=0)
drives['Quarter'] = drives['Quarter'].astype(int)
drives['minute'] = drives['Time'].str.extract(r'([0-9]+):').astype(int)
drives['seconds'] = drives['Time'].str.extract(r'[0-9]+:([0-9]+)').astype(int)
drives['seconds_ratio'] = (drives['seconds'] / 60).astype(float)
drives['Numeric_time'] = drives['minute'] + drives['seconds_ratio']
drives = drives.sort_values(by=['Quarter', 'Numeric_time'], ascending=[True, False]).reset_index()
drives['drive_time'] = cf.elapsed_time(drives['Numeric_time'])
drives['drive_start_time'] = cf.game_time(drives['drive_time'])
drives = drives.drop(columns=['minute', 'seconds', 'seconds_ratio', 'index', '#', 'Numeric_time', 'drive_time'])
# scraping pbp data
pbp_data = sf.scrape_pbp(game_page_soup)
# Setting up receiving and kicking teams
coin_toss = pbp_data.iloc[0]
pbp_data = pbp_data.drop(0)
coin_toss = coin_toss[7]
teams = re.findall(r'\b[A-Z][a-zA-Z]*\b', coin_toss)
match = re.search(r"(\w+)\s+to\s+receive\s+the\s+opening\s+kickoff", coin_toss)
if match:
    receiving_team = match.group(1)
# dropping timeouts
pbp_data = pbp_data[pbp_data['Location'].str.strip() != '']
pbp_data = pbp_data.dropna(subset=['Location'])
print(f'{home_team} vs. {vis_team} \n total plays: {len(pbp_data)}')
# General Cleaning
pbp_data['Quarter'] = [x if x != 'OT' else int(5) for x in pbp_data['Quarter']]
pbp_data['field_side'] = pbp_data['Location'].str.extract(r'([A-Z]+)')
pbp_data['yardline'] = pbp_data['Location'].str.extract(r'([0-9]+)')
pbp_data['yardline'] = pbp_data['yardline'].astype(int)
pbp_data['minute'] = pbp_data['Time'].str.extract(r'([0-9]+):').astype(int)
pbp_data['seconds'] = pbp_data['Time'].str.extract(r'[0-9]+:([0-9]+)').astype(int)
pbp_data['seconds_ratio'] = (pbp_data['seconds'] / 60).astype(float)
pbp_data['Numeric_time'] = pbp_data['minute'] + pbp_data['seconds_ratio']
pbp_data['play_time'] = cf.elapsed_time(pbp_data['Numeric_time'])
pbp_data['play_start_time'] = cf.game_time(pbp_data['play_time'])
pbp_data = pbp_data.drop(columns=['minute', 'seconds', 'seconds_ratio','Numeric_time', 'play_time'])
pbp_data['Play_Type'] = pbp_data['Detail'].apply(cf.play_type)
pbp_data['possession'] = pbp_data['play_start_time'].apply(lambda play_start: cf.determine_possession(play_start, drives))
yards_gained = cf.yards_gained(pbp_data)
pbp_data['Yardage'] = yards_gained
pbp_data = pbp_data.rename(columns={pbp_data.columns[5]: home_vis[team_keys[pbp_data.columns[5]]], 
                                pbp_data.columns[6]: home_vis[team_keys[pbp_data.columns[6]]]})

Jets vs. Bills 
 total plays: 123


In [4]:
pbp_data


,Quarter,Time,Down,ToGo,Location,away,home,Detail,EPB,EPA,field_side,yardline,play_start_time,Play_Type,possession,Yardage
1,1,15:00,,,NYJ 35,0,0,"Greg Zuerlein kicks off 65 yards, touchback.",0.000,0.610,NYJ,35,0.000000,Special Teams,BUF,0.0
2,1,15:00,1,10,BUF 25,0,0,Josh Allen pass complete short right to Stefon...,0.610,1.010,BUF,25,0.000000,Pass,BUF,7.0
3,1,14:34,2,3,BUF 32,0,0,Josh Allen pass complete short left to Deonte ...,1.010,1.400,BUF,32,0.433333,Pass,BUF,5.0
4,1,13:58,1,10,BUF 37,0,0,James Cook up the middle for 3 yards (tackle b...,1.400,1.260,BUF,37,1.033333,Run,BUF,3.0
5,1,13:25,2,7,BUF 40,0,0,James Cook up the middle for 2 yards (tackle b...,1.260,0.830,BUF,40,1.583333,Run,BUF,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,4,0:02,1,10,NYJ 25,16,16,Zach Wilson kneels for -1 yards,0.610,0.000,NYJ,25,59.966667,Run,BUF,0.0
129,5,10:00,1,10,BUF 25,16,16,"Penalty on Spencer Brown: False Start, 5 yards...",0.610,0.280,BUF,25,65.000000,Run,NYJ,5.0
130,5,10:00,1,15,BUF 20,16,16,Josh Allen pass incomplete short right intende...,0.280,-0.610,BUF,20,65.000000,Pass,NYJ,0.0
131,5,9:57,2,15,BUF 20,16,16,James Cook up the middle for 3 yards (tackle b...,-0.610,-0.890,BUF,20,65.050000,Run,BUF,3.0


In [ ]:
drives['Quarter']

In [ ]:
drives['Quarter'] = [x if x!= 'OT' else float(inf) for x in drives['Quarter']]
drives['Quarter'] = drives['Quarter'].astype(int)